## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed(mph),Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,44.62,87,100,6.91,overcast clouds
1,1,Kapaa,US,22.0752,-159.3190,74.12,95,40,10.36,scattered clouds
2,2,Mataura,NZ,-46.1927,168.8643,46.47,91,100,13.42,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,47.86,57,40,5.75,scattered clouds
4,4,Paamiut,GL,61.9940,-49.6678,35.19,87,87,9.22,overcast clouds


In [21]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 75


In [22]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
trip_cities_df = city_data_df.loc[(city_data_df["Max Temp(F)"] <= max_temp) & \
                                       (city_data_df["Max Temp(F)"] >= min_temp)]
trip_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed(mph),Current Description
1,1,Kapaa,US,22.0752,-159.3190,74.12,95,40,10.36,scattered clouds
6,6,Rikitea,PF,-23.1203,-134.9692,70.59,87,100,1.48,moderate rain
9,9,Port Hedland,AU,-20.3167,118.5667,73.45,100,100,5.75,overcast clouds
12,12,Taloqan,AF,36.7361,69.5345,69.67,45,41,4.90,scattered clouds
25,25,Pacific Grove,US,36.6177,-121.9166,66.34,73,0,11.01,haze
32,32,Ilhabela,BR,-23.7781,-45.3581,68.47,64,20,5.17,few clouds
34,34,Oyem,GA,1.5995,11.5793,69.98,97,97,2.86,moderate rain
35,35,Mount Isa,AU,-20.7333,139.5000,65.97,72,0,6.91,clear sky
39,39,Pisco,PE,-13.7000,-76.2167,68.05,83,20,6.91,haze
44,44,Carnarvon,AU,-24.8667,113.6333,70.25,68,0,16.13,clear sky


In [23]:
# 4a. Determine if there are any empty rows.
trip_cities_df.count()

City_ID                136
City                   136
Country                136
Lat                    136
Lng                    136
Max Temp(F)            136
Humidity(%)            136
Cloudiness(%)          136
Wind Speed(mph)        136
Current Description    136
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = trip_cities_df[["City", "Country", "Max Temp(F)", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp(F),Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,74.12,scattered clouds,22.0752,-159.3190,
6,Rikitea,PF,70.59,moderate rain,-23.1203,-134.9692,
9,Port Hedland,AU,73.45,overcast clouds,-20.3167,118.5667,
12,Taloqan,AF,69.67,scattered clouds,36.7361,69.5345,
25,Pacific Grove,US,66.34,haze,36.6177,-121.9166,
32,Ilhabela,BR,68.47,few clouds,-23.7781,-45.3581,
34,Oyem,GA,69.98,moderate rain,1.5995,11.5793,
35,Mount Isa,AU,65.97,clear sky,-20.7333,139.5000,
39,Pisco,PE,68.05,haze,-13.7000,-76.2167,
44,Carnarvon,AU,70.25,clear sky,-24.8667,113.6333,


In [26]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
hotel_df.head(10)

,City,Country,Max Temp(F),Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,74.12,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Rikitea,PF,70.59,moderate rain,-23.1203,-134.9692,People ThankYou
9,Port Hedland,AU,73.45,overcast clouds,-20.3167,118.5667,The Esplanade Hotel
12,Taloqan,AF,69.67,scattered clouds,36.7361,69.5345,Saraisang street
25,Pacific Grove,US,66.34,haze,36.6177,-121.9166,Lovers Point Inn
32,Ilhabela,BR,68.47,few clouds,-23.7781,-45.3581,Ilha Flat Hotel
34,Oyem,GA,69.98,moderate rain,1.5995,11.5793,Commissariat de police
35,Mount Isa,AU,65.97,clear sky,-20.7333,139.5000,ibis Styles Mt Isa Verona
39,Pisco,PE,68.05,haze,-13.7000,-76.2167,Hotel San Isidro Oficial
44,Carnarvon,AU,70.25,clear sky,-24.8667,113.6333,Hospitality Carnarvon


In [27]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   136
Country                136
Max Temp(F)            136
Current Description    136
Lat                    136
Lng                    136
Hotel Name             136
dtype: int64

In [31]:
# 8a. Create the output File (CSV)
output_data_file = ("Vacation_Search/WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp(F)</dt><dd>{Max Temp(F)} </dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp(F)"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))